# Import required libraries

In [ ]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

cudnn.benchmark = True

In [ ]:
from keras.applications import VGG19
cnn_base = VGG19(weights = 'imagenet',
                 include_top = False,
                 input_shape = (64, 64, 3))

In [ ]:
def extract_features(directory, sample_amount):
    features = np.zeros(shape = (sample_amount, 2, 2, 512))
    labels = np.zeros(shape = (sample_amount))
    generator = datagen.flow_from_directory(
        directory, target_size=(64, 64),
        batch_size = 10,
        class_mode = 'binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = cnn.base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i = i + 1
        if i * batch_size>= sample_amount;
            break
    return features, labels

train_features, train_labels = extract_features(train_folder, 540)
validation_features, validation_labels = extract_features(val_folder, 200)
test_features, test_labels = extract_features(test_folder, 180)

train_features = np.reshape(train_features, (540, 2 * 2 * 512))
